In [ ]:
import warnings
warnings.filterwarnings('ignore')
import wradlib as wrl
import matplotlib.pyplot as pl
import matplotlib as mpl
from matplotlib.collections import PatchCollection
from matplotlib.colors import from_levels_and_colors
from matplotlib.path import Path
import matplotlib.patches as patches
import matplotlib.cm as cm
try:
    get_ipython().magic("matplotlib inline")
except:
    pl.ion()
import numpy as np
import datetime as dt
from osgeo import osr
warnings.filterwarnings('ignore')
from satlib import corcor



In [ ]:
# define GPM data set
# ----------------------
gpm_file_path = '/automount/ags/velibor/gpmdata/dprV7/2A.GPM.DPR.V7-20170308.20141007-S015721-E032951.003445.V05A.HDF5'
#gpm_file_path = '/automount/ags/velibor/gpmdata/dprV7/2A.GPM.DPR.V7-20170308.20141008-S084747-E102017.003465.V05A.HDF5'
#gpm_file_path = '/automount/ags/velibor/gpmdata/dprV7/2A.GPM.DPR.V7-20170308.20160601-S171836-E185110.012837.V05A.HDF5'
#gpm_file_path = '/automount/ags/velibor/gpmdata/dprV7/2A.GPM.DPR.V7-20170308.20150330-S224903-E002135.006166.V05A.HDF5'
#gpm_file_path = '/automount/ags/velibor/gpmdata/dprV7/2A.GPM.DPR.V7-20170308.20151216-S014742-E032015.010213.V05A.HDF5'

# define matching ground radar file
# ----------------------
gr_file_path = '/automount/radar-archiv/scans/2014/2014-10/2014-10-07/n_ppi_010deg/2014-10-07--02:37:44,00.mvol'
#gr_file_path = '/automount/radar-archiv/scans/2014/2014-10/2014-10-08/n_ppi_010deg/2014-10-08--09:45:00,00.mvol'
#gr_file_path = '/automount/radar/scans/2016/2016-06/2016-06-01/n_ppi_010deg/2016-06-01--17:59:50,00.mvol'
#gr_file_path = '/automount/radar/scans/2015/2015-03/2015-03-30/n_ppi_010deg/2015-03-30--23:30:03,00.mvol'
#gr_file_path = '/automount/radar/scans/2015/2015-12/2015-12-16/n_ppi_010deg/2015-12-16--02:45:01,00.mvol'

gpm_file = wrl.util.get_wradlib_data_file(gpm_file_path)
gr2gpm_file = wrl.util.get_wradlib_data_file(gr_file_path)

In [ ]:
# Space-born precipitation radar parameters
sr_pars = {"gpm": {
    "zt": 407000.,  # orbital height of GPM                 APPROXIMATION!
    "dr": 125.,      # gate spacing of GPM
    "gr_file": gr2gpm_file,
}}

In [ ]:
# Set parameters for this procedure
bw_sr = 0.71                  # SR beam width
platf = "gpm"                 # SR platform/product: one out of ["gpm", "trmm"]
zt = sr_pars[platf]["zt"]     # SR orbit height (meters)
dr_sr = sr_pars[platf]["dr"]  # SR gate length (meters)
gr_file = sr_pars[platf]["gr_file"]




In [ ]:
def read_gr_boxpol(filename, loaddata=True):
    """
    Read ground radar boxpol scans
    
    in arbeit....
    
    """
    sdate = []
    refl = []
    
    gr_data = wrl.io.read_generic_netcdf(gr_file)
    grdata, grattrs = wrl.io.read_GAMIC_hdf5(gr_file)
    print gr_file
    
    dat_ = gr_data['what']['date']
    #print dat_
    #dat = dat_[0:10]
    #tim = dat_[11:-1]
    #print (dat, tim)
    # '2014-10-07T02:42:02Z'
    #date = dt.datetime.strptime(dat + tim, "%Y-%d-%m%H:%M:%S")
    #source = gr_data['what']['source']

    lon = gr_data['where']['lon']
    lat = gr_data['where']['lat']
    alt = gr_data['where']['height']

    import glob
    c_liste = glob.glob(gr_file.replace('n_ppi_010deg','*ppi*'))
    print c_liste
    ntilt = len(c_liste)
    
    ref_data = [[]]*ntilt
    rho_data = [[]]*ntilt
    zdr_data = [[]]*ntilt


    print ('ntilit:',ntilt)    
    ngate = np.zeros(ntilt, dtype=np.int16)
    nbeam = np.zeros(ntilt)
    elang = np.zeros(ntilt)
    r0 = np.zeros(ntilt)
    dr = np.zeros(ntilt)
    a0 = np.zeros(ntilt)
    
    for jj in range(ntilt):
        print c_liste[jj]
        
        gr_data = wrl.io.read_generic_netcdf(c_liste[jj])
        grdata, grattrs = wrl.io.read_GAMIC_hdf5(c_liste[jj])

        #for i in range(0, ntilt):
        dset = gr_data['scan0']
        # Azimuth Start
        a0[jj] = dset['how']['azi_start'] 
        # Elevations Winkel
        elang[jj] = dset['how']['elevation']
        # Anzahl bins
        ngate[jj] = dset['how']['bin_count']
        # Start und Step der range
        r0[jj] = dset['how']['range_start']
        dr[jj] = dset['how']['range_step']
        # Anzahl Rays
        nbeam[jj] = dset['how']['ray_count']

        #print ('azi start: ', a0)    
        #print ('elevation: ', elang)    
        #print ('bin count: ', ngate)    
        #print ('range start: ', r0)    
        #print ('range step: ', dr)    
        #print ('ray count: ', nbeam)    


        gr_dict = {}
        gr_dict.update({'date': dat_, 'lon': lon, 'lat': lat,
                        'alt': alt, 'ngate': ngate, 'nbeam': nbeam, 'ntilt': ntilt,
                        'r0': r0, 'dr': dr, 'a0': a0, 'elang': elang})
        if not loaddata:
            return gr_dict



        #for i in range(0, ntilt):
            #dset = gr_data['scan0']
            #data = dset['variables']['moment_10']['data']
        #dh = grdata['SCAN0']['ZH']['data']
        #dv = grdata['SCAN0']['ZV']['data']
        #data = (dh + dv)/2.
        data = grdata['SCAN0']['ZH']['data']
        
        rho = grdata['SCAN0']['RHOHV']['data']
        
        zdr = grdata['SCAN0']['ZDR']['data']
        

        #refl.append(data)
        ref_data[jj]=data
        rho_data[jj]=rho
        zdr_data[jj]=zdr
        #ref_data[ref_data<=0]=np.nan

        #refl1 = np.array(refl)
    gr_dict.update({'refl': ref_data})
    gr_dict.update({'rho': rho_data})
    gr_dict.update({'zdr': zdr_data})
    
    return gr_dict, ntilt



In [ ]:
# read matching GR data
gr_data, ntilt = read_gr_boxpol(gr_file)


# Beginn der QnD shitty For schleife

In [ ]:
SR, GR =  np.array([]), np.array([])
X, Y, Z = np.array([]), np.array([]), np.array([])

GR_zdr = np.array([])

gr_nr_elev, gr_ele = np.array([]), np.array([])


for ijk in range(ntilt):
#for ijk in range(1,6,2):

    ######### GR data and att
    ee = ijk
    # number of rays in gr sweep
    nray_gr = gr_data['nbeam'].astype("i4")[ee]
    # number of gates in gr beam
    ngate_gr = gr_data['ngate'].astype("i4")[ee]
    # number of sweeps
    nelev = gr_data['ntilt']
    # elevation of sweep (degree)
    elev_gr = gr_data['elang'][ee]
    # gate length (meters)
    dr_gr = gr_data['dr'][ee]
    # sweep datetime stamp
    #date_gr = gr_data['sdate'][ee]
    # range of first gate
    r0_gr = gr_data['r0'][ee]
    # azimuth angle of first beam
    a0_gr = gr_data['a0'][ee]
    # Longitude of GR
    lon0_gr = gr_data['lon']
    # Latitude of GR
    lat0_gr = gr_data['lat']
    # Altitude of GR (meters)
    alt0_gr = gr_data['alt']
    # Beam width of GR (degree)
    bw_gr = 1.
    # reflectivity array of sweep
    ref_gr = gr_data['refl'][ee]
    # rhohv array of sweep
    rho_gr = gr_data['rho'][ee]
    # zdr array of sweep
    zdr_gr = gr_data['zdr'][ee]
    
    if elev_gr<0.01:
        print ("Elevation zu niedrig")
        pass
    else:
        ############################################################################## Korrektur der GR Daten
        print ("----------------Elevationswinkel: ", elev_gr)
        print ("________Offset______")
        #Hardcoded....passt in etwa
        #ref_gr = ref_gr + 2


        print ("________ATTCORR______")
        
        """pia_harrison = wrl.atten.correctAttenuationHB(
            ref_gr,
            coefficients = dict(a=4.57e-5, b=0.731, gate_length=1.0),
            mode="warn",
            thrs=59.)
        pia_harrison[pia_harrison > 4.8] = 4.8"""

        print ("________ATTCORR2______")
        """ref_gr = ref_gr + pia_harrison"""


        print ("________CLUTTER______")
        rho_th  = 0.85
        ref_gr[rho_gr<= rho_th] = np.nan

        print ("________TH______")
        TH = 15
        ref_gr[ref_gr<=TH]=np.nan
        
        # wichtige ZDR Werte nehmen
        zdr_gr[zdr_gr > 5.]=np.nan
        zdr_gr[zdr_gr < -5.]=np.nan

        print ('----ref_gr: ',np.unique(ref_gr))

        coord = wrl.georef.sweep_centroids(nray_gr, dr_gr, ngate_gr, elev_gr)
        coords = wrl.georef.spherical_to_proj(coord[..., 0],
                                              np.degrees(coord[..., 1]),
                                              coord[..., 2],
                                              (lon0_gr, lat0_gr, alt0_gr))
        lon = coords[..., 0]
        lat = coords[..., 1]
        alt = coords[..., 2]
        bbox = wrl.zonalstats.get_bbox(lon, lat)
        print("Radar bounding box:\n\t%.2f\n%.2f           %.2f\n\t%.2f" %
              (bbox['top'], bbox['left'], bbox['right'], bbox['bottom']))
        sr_data = wrl.io.read_gpm(gpm_file, bbox)

        # Longitudes of SR scans
        sr_lon = sr_data['lon']
        # Latitudes of SR scans
        sr_lat = sr_data['lat']
        # Precip flag
        pflag = sr_data['pflag']
        # Number of scans on SR data
        nscan_sr= sr_data['nscan']
        # Number of rays in one SR scan
        nray_sr = sr_data['nray']
        # Number of gates in one SR ray
        ngate_sr = sr_data['nbin']

        # Calculate equivalent earth radius
        wgs84 = wrl.georef.get_default_projection()
        re1 = wrl.georef.get_earth_radius(lat0_gr, wgs84)
        #print("Earth radius 1:", re1)
        a = wgs84.GetSemiMajor()
        b = wgs84.GetSemiMinor()
        #print("SemiMajor, SemiMinor:", a, b)

        # Set up aeqd-projection gr-centered
        rad = wrl.georef.proj4_to_osr(('+proj=aeqd +lon_0={lon:f} ' +
                                           '+lat_0={lat:f} +a={a:f} ' +
                                           '+b={b:f}').format(lon=lon0_gr,
                                                              lat=lat0_gr,
                                                              a=a, b=b))
        re2 = wrl.georef.get_earth_radius(lat0_gr, rad)
        #print("Earth radius 2:", re2)

        # create gr range and azimuth arrays
        rmax_gr = r0_gr + ngate_gr * dr_gr
        r_gr = np.arange(0, ngate_gr) * dr_gr + dr_gr/2.
        az_gr = np.arange(0, nray_gr) - a0_gr
        #print("Range/Azi-Shape:", r_gr.shape, az_gr.shape)

        # create gr polar grid and calculate aeqd-xyz coordinates
        gr_polargrid = np.meshgrid(r_gr, az_gr)
        gr_xyz, rad = wrl.georef.spherical_to_xyz(gr_polargrid[0], gr_polargrid[1], elev_gr, (lon0_gr, lat0_gr, alt0_gr ))
        #print("XYZ-Grid-Shape:", gr_xyz.shape)

        # create gr poygon array in aeqd-xyz-coordinates
        gr_poly, rad1 = wrl.georef.spherical_to_polyvert(r_gr, az_gr, elev_gr, (lon0_gr, lat0_gr, alt0_gr))
        #print(gr_poly.shape, 360 * 600)
        gr_poly.shape = (nray_gr, ngate_gr, 5, 3)

        # get radar domain (outer ring)
        gr_domain = gr_xyz[:,-1,0:2]
        gr_domain = np.vstack((gr_domain, gr_domain[0]))
        #print("Domain-Shape:", gr_domain.shape)

        sr_x, sr_y = wrl.georef.reproject(sr_lon, sr_lat,
                                          projection_source=wgs84,
                                          projection_target=rad)
        sr_xy = np.dstack((sr_x, sr_y))
        #print("SR-GRID-Shapes:", sr_x.shape, sr_y.shape, sr_xy.shape)

        precip_mask = (pflag >= 2) & wrl.zonalstats.get_clip_mask(sr_xy, gr_domain, rad)

        #print("NRAY", nray_sr)
        #print("NBIN", ngate_sr)

        # use localZenith Angle
        alpha = sr_data['zenith']
        beta = abs(-17.04 + np.arange(nray_sr) * bw_sr)

        # Correct for parallax, get 3D-XYZ-Array
        #   xyzp_sr: Parallax corrected xyz coordinates
        #   r_sr_inv: range array from ground to SR platform
        #   zp: SR bin altitudes
        xyp_sr, r_sr_inv, z_sr = wrl.georef.correct_parallax(sr_xy, ngate_sr, dr_sr, alpha)
        print(xyp_sr.shape, r_sr_inv.shape, z_sr.shape)
        xyzp_sr = np.concatenate((xyp_sr, z_sr[..., np.newaxis]),
                           axis=-1)
        r_sr, az_sr, elev_sr = wrl.georef.xyz_to_spherical(xyzp_sr, alt0_gr, proj=rad)
        #TODO: hardcoded 1.0
        mask = (elev_sr > (1.0 - bw_gr/2.)) & (elev_sr < (1.0 + bw_gr/2.))

        rs = wrl.georef.dist_from_orbit(zt, alpha, beta, r_sr_inv, re1)

        # Small anngle approximation
        vol_sr2  = np.pi * dr_sr * rs**2 * np.radians(bw_sr / 2.)**2

        # Or using wradlib's native function
        vol_sr = wrl.qual.pulse_volume(rs, dr_sr, bw_sr)

        # GR pulse volumes
        #   along one beam
        vol_gr = wrl.qual.pulse_volume(r_gr, dr_gr, bw_gr)
        #   with shape (nray_gr, ngate_gr)
        vol_gr = np.repeat(vol_gr, nray_gr).reshape((nray_gr, ngate_gr), order="F")

        Rs = 0.5 * (1 +  np.cos(np.radians(alpha)))[:,:,np.newaxis] * rs * np.tan(np.radians(bw_sr/2.))
        Ds = dr_sr / np.cos(np.radians(alpha))
        Ds = np.broadcast_to(Ds[..., np.newaxis], Rs.shape)

        ratio, ibb = wrl.qual.get_bb_ratio(sr_data['zbb'], sr_data['bbwidth'], sr_data['quality'], z_sr)

        zbb = sr_data['zbb'].copy()
        zbb[~ibb] = np.nan

        ref_sr = sr_data['refl'].filled(np.nan)


        # First assumption: no valid SR bins (all False)
        valid = np.asarray(elev_sr, dtype=np.bool)==False
        print(valid.shape, precip_mask.shape)

        # SR is inside GR range and is precipitating
        iscan = precip_mask.nonzero()[0]
        iray = precip_mask.nonzero()[1]
        valid[iscan,iray] = True

        # SR bins intersect with GR sweep
        valid = valid & (elev_sr >= (elev_gr-bw_gr/2.)) & (elev_sr <= (elev_gr+bw_gr/2.))

        # Number of matching SR bins per profile
        nvalids = np.sum(valid, axis=2)

        # scan and ray indices for profiles with at least one valid bin
        vscan, vray = np.where(nvalids>0)

        # number of profiles with at least one valid bin
        nprof = len(vscan)

        # average coordinates
        xyz_v1 = xyzp_sr.copy()
        print(xyz_v1.shape)
        xyz_v1[~valid] = np.nan
        xyz_c1 = xyzp_sr.copy()
        xyz_c1[~valid] = 0
        c = np.count_nonzero(xyz_c1[..., 0], axis=2)
        ntotsr = c[vscan, vray]
        xyz_m1 = np.nanmean(xyz_v1,axis=2)
        xyz = xyz_m1[vscan, vray]
        print(xyz.shape, c.shape)

        # approximate Rs
        rs_v1 = Rs.copy()
        rs_v1[~valid] = np.nan
        rs_m1 = np.nanmax(rs_v1, axis=2)
        rs_prof = rs_m1[vscan, vray]
        ds = rs_prof

        # approximate Ds
        ds_v1 = Ds.copy()
        ds_v1[~valid] = np.nan
        ds_m1 = np.nansum(ds_v1, axis=2)
        ds_prof = ds_m1[vscan, vray]
        dz = ds_prof

        # approximate Vs
        vs_v1 = vol_sr.copy()
        vs_v1[~valid] = np.nan
        vs_m1 = np.nansum(vs_v1, axis=2)
        vs_prof = vs_m1[vscan, vray]
        volsr1 = vs_prof

        r_sr, az_sr, el_rs = wrl.georef.xyz_to_spherical(xyz, alt0_gr, proj=rad)

        ref_sr_1 = wrl.trafo.idecibel(ref_sr)
        ref_sr_1[~valid] = np.nan
        refsr1a = np.nanmean(ref_sr_1, axis=2)[vscan,vray]
        refsr1a = wrl.trafo.decibel(refsr1a)


        zds = wrl.zonalstats.DataSource(xyz[:, 0:2].reshape(-1, 2), rad)


        tmp_trg_lyr = zds.ds.GetLayer()
        trg_poly = []
        for i, feat in enumerate(tmp_trg_lyr):
            geom = feat.GetGeometryRef().Buffer(rs_prof[i])
            poly = wrl.georef.ogr_to_numpy(geom)
            trg_poly.append(poly)    

        zdp = wrl.zonalstats.ZonalDataPoly(gr_poly[..., 0:2].reshape(-1, 5, 2), trg_poly, srs=rad)
        zdp.dump_vector('m3d_zonal_poly_{0}'.format(platf))

        obj3 = wrl.zonalstats.GridCellsToPoly('m3d_zonal_poly_{0}'.format(platf))

        print(obj3.ix.shape)
        volgr1 = np.array([np.sum(vol_gr.ravel()[obj3.ix[i]])
                           for i in np.arange(len(obj3.ix))[~obj3.check_empty()]])
        print(volgr1.shape)

        ref_gr_i = wrl.trafo.idecibel(ref_gr.ravel())
        #ref_gr_ku_i = wrl.trafo.idecibel(ref_gr_ku.ravel())
        refgr1a = np.array([np.nanmean(ref_gr_i[obj3.ix[i]])
                     for i in np.arange(len(obj3.ix))[~obj3.check_empty()]])
        refgr1a = wrl.trafo.decibel(refgr1a)
        
        
        #zdr
        zdr_gr_i = wrl.trafo.idecibel(zdr_gr.ravel())
        #ref_gr_ku_i = wrl.trafo.idecibel(ref_gr_ku.ravel())
        zdrgr1a = np.array([np.nanmedian(zdr_gr_i[obj3.ix[i]])
                     for i in np.arange(len(obj3.ix))[~obj3.check_empty()]])
        zdrgr1a = wrl.trafo.decibel(zdrgr1a)
        

        print ('Durchgang: ', ijk)

        pl.figure(figsize=(18,10))
        pl.subplot(1,3,1)
        pl.scatter(xyz[...,0], xyz[...,1], c=refsr1a, label=str(elev_gr),vmin=0, vmax=40)
        pl.colorbar()
        pl.subplot(1,3,2)
        pl.scatter(xyz[...,0], xyz[...,1], c=refgr1a, label=str(elev_gr),vmin=0, vmax=40)
        pl.colorbar()
        pl.subplot(1,3,3)
        pl.scatter(xyz[...,0], xyz[...,1], c=zdrgr1a, label=str(elev_gr))
        pl.colorbar()
        
        #pl.colorbar() 
        #pl.savefig('/automount/ftp/velibor/validation/match_ele'+str(elev_gr)+'.png')
        #pl.close()
        pl.show()
        
        X = np.append(X,xyz[...,0])
        Y = np.append(Y,xyz[...,1])
        Z = np.append(Z,xyz[...,2])

        SR = np.append(SR, refsr1a)
        GR = np.append(GR, refgr1a)
        GR_zdr = np.append(GR_zdr, zdrgr1a)

        gr_nr_elev = np.append(gr_nr_elev, refgr1a.shape)
        gr_ele = np.append(gr_ele, elev_gr)



    

    
    

In [ ]:
from satlib import corcor
from pcc import get_miub_cmap
TH = 15
SR[SR<=TH]=np.nan
GR[GR<=TH]=np.nan


fig = pl.figure(figsize=(14,14))
ax = fig.add_subplot(221, aspect="equal")
pl.scatter(GR, SR, marker="+", c="black")
pl.plot([10,50],[10,50], linestyle="solid", color="black")
#pl.xlim(10,50)
#pl.ylim(10,50)
pl.grid()
pl.xlabel("GR reflectivity (dBZ)")
pl.ylabel("SR reflectivity (dBZ)")
pl.title(corcor(GR, SR))

ax = fig.add_subplot(222)
m0 = ~np.isnan(GR)& ~np.isnan(SR)

pl.hist(GR[m0], bins=20, edgecolor="blue", facecolor="grey", label="GR")
pl.hist(SR[m0], bins=20, edgecolor="red", facecolor="None", label="SR", lw=3)
pl.xlabel("Reflectivity (dBZ)")
pl.legend()
pl.grid()
pl.title('Matched SR and GR Ref')
pl.xlim(10,50)

ax = fig.add_subplot(223)
pl.hist2d(GR[m0], SR[m0],bins=20)

pl.xlabel("GR reflectivity (dBZ)")
pl.ylabel("SR reflectivity (dBZ)")
pl.legend()
pl.grid()
pl.colorbar()
#pl.xlim(10,50)



pl.tight_layout()
pl.savefig('/automount/ftp/velibor/validation/matching1.png')

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = pl.figure(figsize=(18,6))
ax1 = fig.add_subplot(131, projection='3d')
pppl = ax1.scatter(X, Y, Z, c=SR.ravel(), vmin=0, vmax=40, cmap=get_miub_cmap())
fig.colorbar(pppl, shrink=0.5, aspect=5)
ax1.set_title('Ref SR')

ax2 = fig.add_subplot(132, projection='3d')
pppl = ax2.scatter(X, Y, Z, c=GR.ravel(), vmin=0, vmax=40, cmap=get_miub_cmap())
fig.colorbar(pppl, shrink=0.5, aspect=5)
ax2.set_title('Ref GR')

ax3 = fig.add_subplot(133, projection='3d')
pppl = ax3.scatter(X, Y, Z, c=SR.ravel()-GR.ravel(),
                   cmap='coolwarm',vmin=-10,vmax=10)
fig.colorbar(pppl, shrink=0.5, aspect=5)
ax3.set_title('diffRef SR-GR')

from satlib import corcor
print ('Corellation: ',corcor( SR, GR))
pl.savefig('/automount/ftp/velibor/validation/matching2.png')

In [ ]:
fig = pl.figure(figsize=(12,8))
ax = fig.add_subplot(131, aspect="auto")
pl.scatter(Y,Z, c=SR,  vmin=0, vmax=40, edgecolor="None", cmap=get_miub_cmap())
pl.title("SR reflectivity")
pl.xlim(np.nanmin(Y),np.nanmax(Y))
pl.ylim(np.nanmin(Z),8000)
pl.grid()
pl.colorbar(shrink=0.5)

ax = fig.add_subplot(132, aspect="auto")
pl.scatter(Y,Z, c=GR,  vmin=0, vmax=40, edgecolor="None", cmap=get_miub_cmap())
pl.title("GR reflectivity")
pl.xlim(np.nanmin(Y),np.nanmax(Y))
pl.ylim(np.nanmin(Z),8000)
pl.grid()
pl.colorbar(shrink=0.5)
#fig.suptitle("uncorrected GR vs uncorrected SR")
pl.tight_layout()

ax = fig.add_subplot(133, aspect="auto")
pl.scatter(Y,Z, c=SR-GR, cmap='jet',vmin=-10,vmax=10 ,edgecolor="None")
pl.title("SR - GR reflectivity")
pl.xlim(np.nanmin(Y),np.nanmax(Y))
pl.ylim(np.nanmin(Z),8000)
pl.grid()
pl.colorbar(shrink=0.5)
#fig.suptitle("uncorrected SR - GR")
pl.tight_layout()
pl.savefig('/automount/ftp/velibor/validation/matching3.png')

In [ ]:
fig = pl.figure(figsize=(12,8))
ax = fig.add_subplot(131, aspect="auto")
pl.scatter(X,Z, c=SR, cmap=get_miub_cmap(), vmin=0, vmax=40, edgecolor="None")
pl.title("SR reflectivity")
pl.xlim(np.nanmin(X),np.nanmax(X))
pl.ylim(np.nanmin(Z),8000)
pl.grid()
pl.colorbar(shrink=0.5)

ax = fig.add_subplot(132, aspect="auto")
pl.scatter(X,Z, c=GR, cmap=get_miub_cmap(), vmin=0, vmax=40, edgecolor="None")
pl.title("GR reflectivity")
pl.xlim(np.nanmin(X),np.nanmax(X))
pl.ylim(np.nanmin(Z),8000)
pl.grid()
pl.colorbar(shrink=0.5)
#fig.suptitle("uncorrected GR vs uncorrected SR")
pl.tight_layout()

ax = fig.add_subplot(133, aspect="auto")
pl.scatter(X,Z, c=SR-GR, cmap='jet',vmin=-10,vmax=10 ,edgecolor="None")
pl.title("SR - GR reflectivity")
pl.xlim(np.nanmin(X),np.nanmax(X))
pl.ylim(np.nanmin(Z),8000)
pl.grid()
pl.colorbar(shrink=0.5)
#fig.suptitle("uncorrected SR - GR")
pl.tight_layout()
pl.savefig('/automount/ftp/velibor/validation/matching4.png')

In [ ]:
fig = pl.figure(figsize=(12,8))
ax = fig.add_subplot(131, aspect="equal")
pl.scatter(X,Y, c=SR, cmap=get_miub_cmap(), vmin=0, vmax=40, edgecolor="None",s=50)
pl.title("SR reflectivity")
pl.xlim(np.nanmin(X),np.nanmax(X))
pl.ylim(np.nanmin(Y),np.nanmax(Y))
pl.grid()
pl.colorbar(shrink=0.5)

ax = fig.add_subplot(132, aspect="equal")
pl.scatter(X,Y, c=GR, cmap=get_miub_cmap(), vmin=0, vmax=40, edgecolor="None",s=50)
pl.title("GR reflectivity")
pl.xlim(np.nanmin(X),np.nanmax(X))
pl.ylim(np.nanmin(Y),np.nanmax(Y))
pl.grid()
pl.colorbar(shrink=0.5)
#fig.suptitle("uncorrected GR vs uncorrected SR")
pl.tight_layout()

ax = fig.add_subplot(133, aspect="equal")
pl.scatter(X,Y, c=SR-GR, cmap='jet',vmin=-5,vmax=5 ,edgecolor="None",s=50)
pl.title("SR - GR reflectivity")
pl.xlim(np.nanmin(X),np.nanmax(X))
pl.ylim(np.nanmin(Y),np.nanmax(Y))
pl.grid()
pl.colorbar(shrink=0.5)
#fig.suptitle("uncorrected SR - GR")
pl.tight_layout()
pl.savefig('/automount/ftp/velibor/validation/matching5.png')

In [ ]:
def plot_cfad(x1, x2, ZP):
    """
    Plot 4 CFAD
    """
    import matplotlib.pyplot as plt
    m1 = ~np.isnan(x1) & ~np.isnan(x2)
    
    x3 = np.array([]); x4 = np.array([]); x5 = np.array([])

    hh = np.array([])
    for jj in range(0,20000,100):
        hh = np.append(hh,jj)
        x3 = np.append(x3,np.nanmean(x1[np.where((x2>jj)&(x2<jj+100))]))
        x4 = np.append(x4,np.nanmax(x1[np.where((x2>jj)&(x2<jj+100))]))
    
    plt.figure(figsize=(12,12)) 
    plt.hexbin(x1[m1], x2[m1],bins=10, cmap='inferno')
    plt.colorbar()
    plt.xlabel('ZH in DBz')
    plt.ylabel('Height in m')
    plt.title('CFAD: '+ ZP)
    plt.plot(x3,hh, linewidth=2, label='mean')
    plt.plot(x4,hh, linewidth=2, label='median')
    plt.ylim(0,10000)
    plt.xlim(0,60)
    plt.legend(loc='upper right', fontsize=10)
    plt.grid(color='white')
    #plt.close()
    #plt.ylim(0,)
    #plt.show()



In [ ]:
plot_cfad(SR,Z,'SR')
pl.savefig('/automount/ftp/velibor/validation/cfadSR.png')



In [ ]:
plot_cfad(GR,Z,'GR')
pl.savefig('/automount/ftp/velibor/validation/CFADGR.png')


In [ ]:
hhh=0

h = np.arange(0,15000,100)
cor = np.array([])
for ccc in h:
    cor = np.append(cor,corcor(SR[Z>ccc],GR[Z>ccc]))

                    
#       
pl.figure(figsize=(14,14))
pl.subplot(2,2,1)
pl.scatter(SR[Z>hhh],GR[Z>hhh],c=Z[Z>hhh],alpha=0.5)
pl.xlabel('SR in dbz')
pl.ylabel('GR in dbz')
pl.title(corcor(SR[Z>hhh],GR[Z>hhh]))
cb = pl.colorbar()
cb.set_label('h in m')
pl.grid()
            
pl.subplot(2,2,2)
pl.plot(h,cor)
pl.grid()
pl.ylim(0,1)
pl.xlabel('h in m')
pl.ylabel('correlation')

pl.savefig('/automount/ftp/velibor/validation/vertical_corr_Z.png')



In [ ]:
"""aa=0.9

pl.figure(figsize=(16,10))
pl.subplot(2,5,1)
pl.scatter(SR[0:629],GR[0:629],alpha=aa,label='ele: 0.10',color='black')
pl.legend(loc='upper left')
pl.xlabel('SR in dbz')
pl.ylabel('GR in dbz')
pl.title(corcor(SR[0:629],GR[0:629]))
pl.grid()
pl.xlim(0,45)
pl.ylim(0,45)
pl.plot([0,45],[0,45], color='black', ls='--')

pl.subplot(2,5,5)
pl.scatter(SR[629:1254],GR[629:1254],alpha=aa,label='ele: 4.50',color='blue')
pl.legend(loc='upper left')
pl.xlabel('SR in dbz')
pl.ylabel('GR in dbz')
pl.title(corcor(SR[629:1254],GR[629:1254]))
pl.grid()
pl.xlim(0,45)
pl.ylim(0,45)
pl.plot([0,45],[0,45], color='black', ls='--')

pl.subplot(2,5,6)
pl.scatter(SR[1254:2379],GR[1254:2379],alpha=aa,label='ele: 8.20',color='green')
pl.legend(loc='upper left')
pl.xlabel('SR in dbz')
pl.ylabel('GR in dbz')
pl.title(corcor(SR[1254:2379],GR[1254:2379]))
pl.grid()
pl.xlim(0,45)
pl.ylim(0,45)
pl.plot([0,45],[0,45], color='black', ls='--')

pl.subplot(2,5,7)
pl.scatter(SR[2379:2676],GR[2379:2676],alpha=aa,label='ele: 11.0',color='tan')
pl.legend(loc='upper left')
pl.xlabel('SR in dbz')
pl.ylabel('GR in dbz')
pl.title(corcor(SR[2379:2676],GR[2379:2676]))
pl.grid()
pl.xlim(0,45)
pl.ylim(0,45)
pl.plot([0,45],[0,45], color='black', ls='--')

pl.subplot(2,5,8)
pl.scatter(SR[2676:2861],GR[2676:2861],alpha=aa,label='ele: 14.0',color='red')
pl.legend(loc='upper left')
pl.xlabel('SR in dbz')
pl.ylabel('GR in dbz')
pl.title(corcor(SR[2676:2861],GR[2676:2861]))
pl.grid()
pl.xlim(0,45)
pl.ylim(0,45)
pl.plot([0,45],[0,45], color='black', ls='--')

pl.subplot(2,5,9)
pl.scatter(SR[2861:2944],GR[2861:2944],alpha=aa,label='ele: 18.0',color='gray')
pl.legend(loc='upper left')
pl.xlabel('SR in dbz')
pl.ylabel('GR in dbz')
pl.title(corcor(SR[2861:2944],GR[2861:2944]))
pl.grid()
pl.xlim(0,45)
pl.ylim(0,45)
pl.plot([0,45],[0,45], color='black', ls='--')

pl.subplot(2,5,10)
pl.scatter(SR[2944:2950],GR[2944:2950],alpha=aa,label='ele: 28.0',color='orange')
pl.legend(loc='upper left')
pl.xlabel('SR in dbz')
pl.ylabel('GR in dbz')
pl.title(corcor(SR[2944:2950],GR[2944:2950]))
pl.grid()
pl.xlim(0,45)
pl.ylim(0,45)
pl.plot([0,45],[0,45], color='black', ls='--')

pl.subplot(2,5,2)
pl.scatter(SR[2950:3252],GR[2950:3252],alpha=aa,label='ele: 1.5',color='brown')
pl.legend(loc='upper left')
pl.xlabel('SR in dbz')
pl.ylabel('GR in dbz')
pl.title(corcor(SR[2950:3252],GR[2950:3252]))
pl.grid()
pl.xlim(0,45)
pl.ylim(0,45)
pl.plot([0,45],[0,45], color='black', ls='--')

pl.subplot(2,5,3)
pl.scatter(SR[3252:3554],GR[3252:3554],alpha=aa,label='ele: 2.5',color='cyan')
pl.legend(loc='upper left')
pl.xlabel('SR in dbz')
pl.ylabel('GR in dbz')
pl.title(corcor(SR[3252:3554],GR[3252:3554]))
pl.xlim(0,45)
pl.ylim(0,45)
pl.plot([0,45],[0,45], color='black', ls='--')
pl.grid()

pl.subplot(2,5,4)
pl.scatter(SR[3554:3854],GR[3554:3854],alpha=aa,label='ele: 3.4',color='magenta')
pl.legend(loc='upper left')
pl.xlabel('SR in dbz')
pl.ylabel('GR in dbz')
pl.title(corcor(SR[3554:3854],GR[3554:3854]))
pl.xlim(0,45)
pl.ylim(0,45)
pl.plot([0,45],[0,45], color='black', ls='--')

pl.grid()
pl.savefig('/automount/ftp/velibor/validation/corr_elev.png')"""

In [ ]:
"""aa=0.9
from pcc import get_miub_cmap
pl.figure(figsize=(30,10))
pl.subplot(2,5,1)
pl.scatter(X[0:629],Y[0:629],c=GR[0:629], cmap=get_miub_cmap(), vmin=0, vmax=40)
pl.grid()
pl.colorbar()
pl.xlim(-150000,150000)
pl.ylim(-150000,150000)


pl.subplot(2,5,5)
pl.scatter(X[629:1254],Y[629:1254],c=GR[629:1254], cmap=get_miub_cmap(), vmin=0, vmax=40)
pl.grid()
pl.colorbar()
pl.xlim(-150000,150000)
pl.ylim(-150000,150000)

pl.subplot(2,5,6)
pl.scatter(X[1254:2379],Y[1254:2379],c=GR[1254:2379], cmap=get_miub_cmap(), vmin=0, vmax=40)
pl.grid()
pl.colorbar()
pl.xlim(-150000,150000)
pl.ylim(-150000,150000)

pl.subplot(2,5,7)
pl.scatter(X[2379:2676],Y[2379:2676],c=GR[2379:2676], cmap=get_miub_cmap(), vmin=0, vmax=40)
pl.grid()
pl.colorbar()
pl.xlim(-150000,150000)
pl.ylim(-150000,150000)

pl.subplot(2,5,8)
pl.scatter(X[2676:2861],Y[2676:2861],c=GR[2676:2861], cmap=get_miub_cmap(), vmin=0, vmax=40)
pl.grid()
pl.colorbar()
pl.xlim(-150000,150000)
pl.ylim(-150000,150000)

pl.subplot(2,5,9)
pl.scatter(X[2861:2944],Y[2861:2944],c=GR[2861:2944], cmap=get_miub_cmap(), vmin=0, vmax=40)
pl.grid()
pl.colorbar()
pl.xlim(-150000,150000)
pl.ylim(-150000,150000)

pl.subplot(2,5,10)
pl.scatter(X[2944:2950],Y[2944:2950],c=GR[2944:2950], cmap=get_miub_cmap(), vmin=0, vmax=40)
pl.grid()
pl.colorbar()
pl.xlim(-150000,150000)
pl.ylim(-150000,150000)

pl.subplot(2,5,2)
pl.scatter(X[2950:3252],Y[2950:3252],c=GR[2950:3252], cmap=get_miub_cmap(), vmin=0, vmax=40)
pl.grid()
pl.colorbar()
pl.xlim(-150000,150000)
pl.ylim(-150000,150000)


pl.subplot(2,5,3)
pl.scatter(X[3252:3554],Y[3252:3554],c=GR[3252:3554], cmap=get_miub_cmap(), vmin=0, vmax=40)
pl.grid()
pl.colorbar()
pl.xlim(-150000,150000)
pl.ylim(-150000,150000)


pl.subplot(2,5,4)
pl.scatter(X[3554:3854],Y[3554:3854],c=GR[3554:3854], cmap=get_miub_cmap(), vmin=0, vmax=40)
pl.colorbar()
pl.xlim(-150000,150000)
pl.ylim(-150000,150000)
pl.grid()
pl.tight_layout()
pl.savefig('/automount/ftp/velibor/validation/corr_elev_ppi.png')"""

In [ ]:
mm = ~np.isnan(GR)& ~np.isnan(SR)
pl.figure(figsize=(14,10))

pl.hist(GR[mm]-SR[mm],bins=100, facecolor="grey", alpha=0.6)
pl.axvline(np.mean(GR[mm]-SR[mm]), color='blue', label='Mean diff')
pl.axvline(np.median(GR[mm]-SR[mm]), color='green', label='Median diff')
pl.axvline(0, color='black',ls='--',lw=3)

xmin, xmax = np.nanmin(GR[mm]-SR[mm]), np.nanmax(GR[mm]-SR[mm])
limits = np.nanmax([abs(xmin),abs(xmax)])
pl.xlim(-limits,limits)

pl.title('Diff GR-SR, bias: '+str( np.sum(GR[mm]-SR[mm])/len(GR[mm]-SR[mm]))+' dbz')
pl.grid()
pl.legend()
pl.xlabel('SR overestimate----------------------------Diff in dbz----------------------------GR overestimate')
pl.ylabel('number')
pl.savefig('/automount/ftp/velibor/validation/matching_bias.png')

In [ ]:

m1 = ~np.isnan(GR)
m2 = ~np.isnan(SR)

pl.figure(figsize=(14,5))
pl.hist(SR[mm],bins=50, facecolor="None", alpha=1, edgecolor='black', histtype='step', label='SR')
pl.xlim(left=0)


pl.hist(GR[mm],bins=50, facecolor="None", alpha=1, edgecolor='blue', histtype='step', label='GR')
pl.axvline(0, color='black',ls='--',lw=3)
pl.grid()
pl.legend(fontsize=20)
#pl.ylim(top=90)
pl.ylabel('number', fontsize=20)
pl.xlabel(r'$Z_H$ in dbz', fontsize=20)
pl.xticks(fontsize=20); pl.yticks(fontsize=20)
pl.savefig('/automount/ftp/velibor/validation/matching_hist.png')

In [ ]:
aa=0.9
# Elevation unterteilen mit Cumulativer Summe
e_pos = gr_nr_elev.cumsum().astype(int)

pl.figure(figsize=(16,10))

for eee in range(len(gr_nr_elev)):
    pl.subplot(2,5,eee+1)
    
    if eee==0: 
        pl.scatter(SR[0:e_pos[0]],GR[0:e_pos[0]],alpha=aa,label='elevation: '+str(gr_ele[0])[0:5],color='black',s=10)
        pl.title('r: ' + corcor(SR[0:e_pos[0]],GR[0:e_pos[0]])[0:4])
        pl.legend(loc='upper left')
    else:
        pl.scatter(SR[e_pos[eee-1]:e_pos[eee]],GR[e_pos[eee-1]:e_pos[eee]],alpha=aa,
                   label='elevation: ' + str(gr_ele[eee])[0:5],color='black',s=10)
        pl.title('r: ' + corcor(SR[e_pos[eee-1]:e_pos[eee]],GR[e_pos[eee-1]:e_pos[eee]])[0:6])
        pl.legend(loc='upper left')
    pl.xlabel('SR in dbz')
    pl.ylabel('GR in dbz')
    pl.grid()
    pl.xlim(0,45)
    pl.ylim(0,45)
    pl.plot([0,45],[0,45], color='black', ls='--')

pl.tight_layout()
pl.savefig('/automount/ftp/velibor/validation/corr_elev.png')

In [ ]:
# Elevation unterteilen mit Cumulativer Summe
e_pos = gr_nr_elev.cumsum().astype(int)
from pcc import get_miub_cmap

pl.figure(figsize=(30,10))

for eee in range(len(gr_nr_elev)):
    pl.subplot(2,5,eee+1)
    
    if eee==0: 
        pl.scatter(X[0:e_pos[0]],Y[0:e_pos[0]],c=GR[0:e_pos[0]],label=str(gr_ele[0]),
                   cmap=get_miub_cmap(), vmin=0, vmax=40)
        pl.legend(loc='upper left')
    else:
        pl.scatter(X[e_pos[eee-1]:e_pos[eee]],Y[e_pos[eee-1]:e_pos[eee]],c=GR[e_pos[eee-1]:e_pos[eee]],
                   label=str(gr_ele[eee]), cmap=get_miub_cmap(), vmin=0, vmax=40)
        pl.legend(loc='upper left')
    pl.xlabel('SR in dbz')
    pl.ylabel('GR in dbz')
    pl.grid()
    pl.xlim(-150000,150000)
    pl.ylim(-150000,150000)
    pl.colorbar()


pl.tight_layout()
pl.savefig('/automount/ftp/velibor/validation/corr_ppiGR.png')



In [ ]:
# Elevation unterteilen mit Cumulativer Summe
e_pos = gr_nr_elev.cumsum().astype(int)
from pcc import get_miub_cmap

pl.figure(figsize=(30,10))

for eee in range(len(gr_nr_elev)):
    pl.subplot(2,5,eee+1)
    
    if eee==0: 
        pl.scatter(X[0:e_pos[0]],Y[0:e_pos[0]],c=SR[0:e_pos[0]],label=str(gr_ele[0]),
                   cmap=get_miub_cmap(), vmin=0, vmax=40)
        pl.legend(loc='upper left')
    else:
        pl.scatter(X[e_pos[eee-1]:e_pos[eee]],Y[e_pos[eee-1]:e_pos[eee]],c=SR[e_pos[eee-1]:e_pos[eee]],
                   label=str(gr_ele[eee]), cmap=get_miub_cmap(), vmin=0, vmax=40)
        pl.legend(loc='upper left')
    pl.xlabel('SR in dbz')
    pl.ylabel('GR in dbz')
    pl.grid()
    pl.xlim(-150000,150000)
    pl.ylim(-150000,150000)
    pl.colorbar()


pl.tight_layout()
pl.savefig('/automount/ftp/velibor/validation/corr_ppiSR.png')



In [ ]:
# Elevation unterteilen mit Cumulativer Summe
e_pos = gr_nr_elev.cumsum().astype(int)
from pcc import get_miub_cmap

pl.figure(figsize=(30,10))

for eee in range(len(gr_nr_elev)):
    pl.subplot(2,5,eee+1)
    
    if eee==0: 
        pl.scatter(X[0:e_pos[0]],Z[0:e_pos[0]],c=SR[0:e_pos[0]],label=str(gr_ele[0]),
                   cmap=get_miub_cmap(), vmin=0, vmax=40)
        pl.legend(loc='upper left')
    else:
        pl.scatter(X[e_pos[eee-1]:e_pos[eee]],Z[e_pos[eee-1]:e_pos[eee]],c=SR[e_pos[eee-1]:e_pos[eee]],
                   label=str(gr_ele[eee]), cmap=get_miub_cmap(), vmin=0, vmax=40)
        pl.legend(loc='upper left')
    pl.xlabel('SR in dbz')
    pl.ylabel('GR in dbz')
    pl.grid()
    pl.xlim(-150000,150000)
    pl.ylim(0,7000)
    pl.colorbar()


pl.tight_layout()
pl.savefig('/automount/ftp/velibor/validation/corr_XppiSR.png')



In [ ]:
# Elevation unterteilen mit Cumulativer Summe
e_pos = gr_nr_elev.cumsum().astype(int)
from pcc import get_miub_cmap

pl.figure(figsize=(30,10))

for eee in range(len(gr_nr_elev)):
    pl.subplot(2,5,eee+1)
    
    if eee==0: 
        pl.scatter(X[0:e_pos[0]],Z[0:e_pos[0]],c=GR[0:e_pos[0]],label=str(gr_ele[0]),
                   cmap=get_miub_cmap(), vmin=0, vmax=40)
        pl.legend(loc='upper left')
    else:
        pl.scatter(X[e_pos[eee-1]:e_pos[eee]],Z[e_pos[eee-1]:e_pos[eee]],c=GR[e_pos[eee-1]:e_pos[eee]],
                   label=str(gr_ele[eee]), cmap=get_miub_cmap(), vmin=0, vmax=40)
        pl.legend(loc='upper left')
    pl.xlabel('SR in dbz')
    pl.ylabel('GR in dbz')
    pl.grid()
    pl.xlim(-150000,150000)
    pl.ylim(0,7000)
    pl.colorbar()


pl.tight_layout()
pl.savefig('/automount/ftp/velibor/validation/corr_XppiGR.png')



In [ ]:
np.nanmin(ref_sr)

In [ ]:
# Elevation unterteilen mit Cumulativer Summe
e_pos = gr_nr_elev.cumsum().astype(int)
from pcc import get_miub_cmap

pl.figure(figsize=(30,10))

for eee in range(len(gr_nr_elev)):
    pl.subplot(2,5,eee+1)
    
    if eee==0: 
        pl.scatter(X[0:e_pos[0]],Y[0:e_pos[0]],c=SR[0:e_pos[0]]-GR[0:e_pos[0]],label=str(gr_ele[0]),
                   cmap='jet', vmin=-10, vmax=10)
        pl.legend(loc='upper left')
    else:
        pl.scatter(X[e_pos[eee-1]:e_pos[eee]],Y[e_pos[eee-1]:e_pos[eee]],c=SR[e_pos[eee-1]:e_pos[eee]]-GR[e_pos[eee-1]:e_pos[eee]],
                   label=str(gr_ele[eee]), cmap='jet', vmin=-10, vmax=10)
        pl.legend(loc='upper left')
    pl.xlabel('X')
    pl.ylabel('Y')
    pl.title('SR-GR')
    pl.grid()
    pl.xlim(-150000,150000)
    pl.ylim(-150000,150000)
    pl.colorbar()


pl.tight_layout()
pl.savefig('/automount/ftp/velibor/validation/corr_ppiSR.png')



In [ ]:
# Elevation unterteilen mit Cumulativer Summe
e_pos = gr_nr_elev.cumsum().astype(int)
from pcc import get_miub_cmap

pl.figure(figsize=(30,10))

for eee in range(len(gr_nr_elev)):
    pl.subplot(2,5,eee+1)
    
    if eee==0: 
        pl.scatter(X[0:e_pos[0]],Z[0:e_pos[0]],c=SR[0:e_pos[0]]-GR[0:e_pos[0]],label=str(gr_ele[0]),
                   cmap='jet', vmin=-10, vmax=10)
        pl.legend(loc='upper left')
    else:
        pl.scatter(X[e_pos[eee-1]:e_pos[eee]],Z[e_pos[eee-1]:e_pos[eee]],c=SR[e_pos[eee-1]:e_pos[eee]]-GR[e_pos[eee-1]:e_pos[eee]],
                   label=str(gr_ele[eee]), cmap='jet', vmin=-10, vmax=10)
        pl.legend(loc='upper left')
    pl.xlabel('X')
    pl.ylabel('Z')
    pl.grid()
    pl.xlim(-150000,150000)
    pl.ylim(0,7000)
    pl.colorbar()


pl.tight_layout()
pl.savefig('/automount/ftp/velibor/validation/corr_XppiSR.png')



In [ ]:
SRi, GRi = SR.copy(), GR.copy()

SRi[SRi>36]=np.nan
SRi[SRi<24]=np.nan
GRi[GRi>36]=np.nan
GRi[GRi<24]=np.nan




fig = pl.figure(figsize=(14,14))
ax = fig.add_subplot(221, aspect="equal")
pl.scatter(GRi, SRi, marker="+", c="black")
pl.plot([10,50],[10,50], linestyle="solid", color="black")
#pl.xlim(10,50)
#pl.ylim(10,50)
pl.grid()
pl.xlabel("GR reflectivity (dBZ)")
pl.ylabel("SR reflectivity (dBZ)")
pl.title(corcor(GRi, SRi))

ax = fig.add_subplot(222)
m0i = ~np.isnan(GRi)& ~np.isnan(SRi)

pl.hist(GRi[m0i], bins=20, edgecolor="blue", facecolor="None", label="GR", histtype='step')
pl.hist(SRi[m0i], bins=20, edgecolor="red", facecolor="None", label="SR", lw=3, histtype='step')
pl.xlabel("Reflectivity (dBZ)")
pl.legend()
pl.grid()
pl.title('Matched SR and GR Ref')

ax = fig.add_subplot(223)
pl.hist2d(GRi[m0i], SRi[m0i],bins=20)

pl.xlabel("GR reflectivity (dBZ)")
pl.ylabel("SR reflectivity (dBZ)")
pl.legend()
pl.grid()
pl.colorbar()
#pl.xlim(10,50)



ax = fig.add_subplot(224)

mmi = ~np.isnan(GRi)& ~np.isnan(SRi)

pl.hist(SRi[mmi]-GRi[mmi],bins=50, facecolor="grey", alpha=0.6)
pl.axvline(np.mean(SRi[mmi]-GRi[mmi]), color='blue', label='Mean diff')
pl.axvline(np.median(SRi[mmi]-GRi[mmi]), color='green', label='Median diff')
pl.axvline(0, color='black',ls='--',lw=3)

xmin, xmax = np.nanmin(SRi[mmi]-GRi[mmi]), np.nanmax(SRi[mmi]-GRi[mmi])
limits = np.nanmax([abs(xmin),abs(xmax)])
pl.xlim(-limits,limits)

pl.title('Diff SR-GR, bias: '+str( np.sum(SRi[mmi]-GRi[mmi])/len(SRi[mmi]-GRi[mmi]))+' dbz')
pl.grid()
pl.legend()
pl.xlabel('GR overestimate----------------------------Diff in dbz----------------------------SR overestimate')
pl.ylabel('number')


In [ ]:
print ('von',mmi.shape, 'Pixel matschen sich ', sum(mmi==True))

In [ ]:
print sum(~np.isnan(GR)), sum(~np.isnan(SR))

In [ ]:
150./600.


In [ ]:

pl.plot(sr_data['zbb']-sr_data['bbwidth'])

In [ ]:
def subplot_cfad(x1, x2, ZP):
    """
    Plot 4 CFAD
    """
    import matplotlib.pyplot as plt
    m1 = ~np.isnan(x1) & ~np.isnan(x2)
    
    x3 = np.array([]); x4 = np.array([]); x5 = np.array([])

    hh = np.array([])
    for jj in range(0,20000,100):
        hh = np.append(hh,jj)
        x3 = np.append(x3,np.nanmean(x1[np.where((x2>jj)&(x2<jj+100))]))
        x4 = np.append(x4,np.nanmedian(x1[np.where((x2>jj)&(x2<jj+100))]))
    
    plt.hist2d(x1[m1], x2[m1],bins=30, cmap='inferno')
    #plt.hexbin(x1[m1], x2[m1],bins=10, cmap='inferno')
    plt.colorbar()
    plt.xlabel('ZH in DBz')
    plt.ylabel('Height in m')
    plt.title('CFAD: '+ ZP)
    plt.plot(x3,hh, linewidth=2, label='mean')
    plt.plot(x4,hh, linewidth=2, label='median')

    plt.legend(loc='upper right', fontsize=10)
    plt.grid(color='white')
    #plt.close()
    #plt.ylim(0,)
    #plt.show()



In [ ]:
diff_SRGR = SR - GR
pl.figure(figsize=(12,6)) 
pl.subplot(1,3,1)
subplot_cfad(SR,Z,'SR')
#pl.xlim(15,45)

pl.subplot(1,3,2)
subplot_cfad(GR,Z,'GR')
#pl.xlim(15,45)

pl.subplot(1,3,3)
subplot_cfad(diff_SRGR,Z,'SR - GR')
pl.axvline(0, color='white')
#pl.xlim(-15,20)
pl.tight_layout()
pl.savefig('/automount/ftp/velibor/validation/cfadSR-GR.pdf')

In [ ]:
pl.figure(figsize=(12,6)) 
pl.subplot(1,3,1)
subplot_cfad(sr_data['refl'].reshape(28* 49* 176),xyzp_sr[:,:,:,2].reshape(28* 49* 176),'SR_real')

In [ ]:
print xyz_v1[:,:,:,2].shape
print sr_data['refl'].shape
print xyzp_sr[:,:,:,2].shape

In [ ]:
A = np.nanmean(sr_data['refl'], axis=(0,1))
h = np.arange(0,176,1)*250.
h = h-3000.
print A.shape, h.shape

pl.plot(A,h)

In [ ]:
A


In [ ]:


# Distanz Berechnen
x1, x2 = 0, X
y1, y2 = 0, Y

dist = np.hypot(x2 - x1, y2 - y1)

#plt.pcolormesh(dpr_lon, dpr_lat, Z_boxpol_ipoli)
pl.scatter(X, Y, c=dist)
pl.colorbar()


pl.figure(figsize=(18,16))
sc = pl.scatter(SR,GR,c=Z, marker='o', s=3**(dist/10000), cmap='spectral')
sc.set_facecolor("none")
pl.plot([15,50],[15,50])
pl.colorbar()
pl.grid()
pl.xlim(15,50); pl.ylim(15,50)
pl.xlabel('SR in dbz', fontsize=18); pl.ylabel('GR in dbz', fontsize=18)
#pl.savefig('/automount/ftp/velibor/validation/007elev.pdf')

In [ ]:
print abs(np.arange(0,15,1)-15)
print np.arange(0,15,1)

In [ ]:


# Distanz Berechnen
x1, x2 = 0, X
y1, y2 = 0, Y

dist = np.hypot(x2 - x1, y2 - y1)

#plt.pcolormesh(dpr_lon, dpr_lat, Z_boxpol_ipoli)
pl.scatter(X, Y, c=dist)
pl.colorbar()


pl.figure(figsize=(18,16))
sc = pl.scatter(SR,GR,c=GR_zdr, marker='o', s=dist/1000, cmap='jet')
sc.set_facecolor("none")
pl.plot([15,50],[15,50])
pl.colorbar()
pl.grid()
pl.xlim(15,50); pl.ylim(15,50)
pl.xlabel('SR in dbz', fontsize=18); pl.ylabel('GR in dbz', fontsize=18)
pl.savefig('/automount/ftp/velibor/validation/007elev.pdf')

In [ ]:
m = ~np.isnan(GR_zdr)
pl.hist(GR_zdr[m])
pl.yscale('log')

In [ ]:
1e5 * 1e4

In [ ]:
(1./5e9) * ((-8.*5e4) - (4./np.cos(np.deg2rad(30))*1e5 - 10.*5e4 + 4.*1e5/np.cos(np.deg2rad(30))))

In [ ]:
(1./5e9) * ((-9.*5e4) - (4./np.cos(np.deg2rad(30))*1e5 - 9.*5e4 + 4.*1e5/np.cos(np.deg2rad(30))))

In [ ]:
0.000094 *3600

In [ ]:


#bounds2 = np.array([-3,0,1,1.5,2.5,3,3.5,4,4.5])

print (bounds2)

# Distanz Berechnen
x1, x2 = 0, X
y1, y2 = 0, Y

dist = np.hypot(x2 - x1, y2 - y1)

v1, v2 =0, 4000 
bounds2 = np.linspace(np.nanmin(Z), np.nanmax(Z), 20)
bounds2 = np.arange(v1,v2, 1000)

#plt.pcolormesh(dpr_lon, dpr_lat, Z_boxpol_ipoli)
pl.scatter(X, Y, c=dist)
pl.colorbar()
#pl.colorbar(shrink=0.8,extend='both',ticks=bounds2, boundaries=bounds2)

pl.figure(figsize=(18,16))
sc = pl.scatter(SR,GR,c=Z, marker='o', s=dist/1000, cmap='cool',vmin = 0, vmax = 4000)
sc.set_facecolor("none")
pl.plot([15,50],[15,50])
pl.colorbar(shrink=0.8,extend='both')
pl.grid()
pl.xlim(15,50); pl.ylim(15,50)
pl.xlabel('SR in dbz', fontsize=18); pl.ylabel('GR in dbz', fontsize=18)

In [ ]:


#bounds2 = np.array([-3,0,1,1.5,2.5,3,3.5,4,4.5])

print (bounds2)

# Distanz Berechnen
x1, x2 = 0, X
y1, y2 = 0, Y

dist = np.hypot(x2 - x1, y2 - y1)

v1, v2 =0, 4000 
bounds2 = np.linspace(np.nanmin(Z), np.nanmax(Z), 20)
bounds2 = np.arange(v1,v2, 1000)

#plt.pcolormesh(dpr_lon, dpr_lat, Z_boxpol_ipoli)
pl.scatter(X, Y, c=dist)
pl.colorbar()
#pl.colorbar(shrink=0.8,extend='both',ticks=bounds2, boundaries=bounds2)

pl.figure(figsize=(18,16))
sc = pl.scatter(SR,GR,c=GR_zdr, marker='o', s=Z/100, cmap='jet', vmin=np.nanmin(GR_zdr), vmax=3)
sc.set_facecolor("none")
pl.plot([15,50],[15,50])
pl.colorbar(shrink=0.8,extend='both')
pl.grid()
pl.xlim(15,50); pl.ylim(15,50)
pl.xlabel('SR in dbz', fontsize=18); pl.ylabel('GR in dbz', fontsize=18)

In [ ]:

pl.figure(figsize=(18,16))
sc = pl.scatter(SR,GR,c=GR_zdr, marker='o', s=80, cmap='jet')
pl.colorbar()

#pl.colorbar(shrink=0.8,extend='both',ticks=bounds2, boundaries=bounds2)

pl.grid()

pl.xlabel('SR'); pl.ylabel('GR')


In [ ]:
aa=0.9
# Elevation unterteilen mit Cumulativer Summe
e_pos = gr_nr_elev.cumsum().astype(int)

pl.figure(figsize=(20,10))

for eee in range(len(gr_nr_elev)):
    pl.subplot(2,5,eee+1)
    
    if eee==0: 
        pl.scatter(SR[0:e_pos[0]],GR[0:e_pos[0]],c=Z[0:e_pos[0]],alpha=aa,
                   label='elevation: '+str(gr_ele[0])[0:5],color='black',s=10, vmin=0, vmax=4000)
        pl.title('r: ' + corcor(SR[0:e_pos[0]],GR[0:e_pos[0]])[0:4])
        pl.legend(loc='upper left')
        pl.colorbar()
    else:
        pl.scatter(SR[e_pos[eee-1]:e_pos[eee]],GR[e_pos[eee-1]:e_pos[eee]],c=Z[e_pos[eee-1]:e_pos[eee]],alpha=aa,
                   label='elevation: ' + str(gr_ele[eee])[0:5],color='black',s=10, vmin=0, vmax=4000)
        pl.title('r: ' + corcor(SR[e_pos[eee-1]:e_pos[eee]],GR[e_pos[eee-1]:e_pos[eee]])[0:6])
        pl.legend(loc='upper left')
        pl.colorbar()
    pl.xlabel('SR in dbz')
    pl.ylabel('GR in dbz')
    pl.grid()
    pl.xlim(0,45)
    pl.ylim(0,45)
    pl.plot([0,45],[0,45], color='black', ls='--')

pl.tight_layout()
pl.savefig('/automount/ftp/velibor/validation/ZDRcorr_elev.pdf')

In [ ]:
np.unique(GR_zdr)

In [ ]:
fig = pl.figure(figsize=(12,8))
ax = fig.add_subplot(131, aspect="auto")
pl.scatter(Y,Z, c=SR,  vmin=0, vmax=40, edgecolor="None", cmap=get_miub_cmap())
pl.title("SR reflectivity")
pl.xlim(np.nanmin(Y),np.nanmax(Y))
pl.ylim(np.nanmin(Z),8000)
pl.grid()
pl.colorbar(shrink=0.5)

ax = fig.add_subplot(132, aspect="auto")
pl.scatter(Y,Z, c=GR,  vmin=0, vmax=40, edgecolor="None", cmap=get_miub_cmap())
pl.title("GR reflectivity")
pl.xlim(np.nanmin(Y),np.nanmax(Y))
pl.ylim(np.nanmin(Z),8000)
pl.grid()
pl.colorbar(shrink=0.5)
#fig.suptitle("uncorrected GR vs uncorrected SR")
pl.tight_layout()

ax = fig.add_subplot(133, aspect="auto")
pl.scatter(Y,Z, c=GR_zdr, cmap='jet',vmin=-1,vmax=3 ,edgecolor="None")
pl.title("ZDR reflectivity")
pl.xlim(np.nanmin(Y),np.nanmax(Y))
pl.ylim(np.nanmin(Z),8000)
pl.grid()
pl.colorbar(shrink=0.5)
#fig.suptitle("uncorrected SR - GR")
pl.tight_layout()
pl.savefig('/automount/ftp/velibor/validation/matching3.png')

In [ ]:
zbb.shape

In [ ]:
28*49

In [ ]:
aa=0.9
# Elevation unterteilen mit Cumulativer Summe
e_pos = gr_nr_elev.cumsum().astype(int)

pl.figure(figsize=(16,10))

for eee in range(len(gr_nr_elev)):
    pl.subplot(2,5,eee+1)
    
    if eee==0: 
        pl.scatter(SR[0:e_pos[0]],GR[0:e_pos[0]],c=GR_zdr[0:e_pos[0]],alpha=aa,label='elevation: '+str(gr_ele[0])[0:5],color='black',s=10, vmin=-0.2, vmax=2)
        pl.title('r: ' + corcor(SR[0:e_pos[0]],GR[0:e_pos[0]])[0:4])
        pl.legend(loc='upper left')
        pl.colorbar()
    else:
        pl.scatter(SR[e_pos[eee-1]:e_pos[eee]],GR[e_pos[eee-1]:e_pos[eee]],c=GR_zdr[e_pos[eee-1]:e_pos[eee]],alpha=aa,
                   label='elevation: ' + str(gr_ele[eee])[0:5],color='black',s=10, vmin=-0.2, vmax=2)
        pl.title('r: ' + corcor(SR[e_pos[eee-1]:e_pos[eee]],GR[e_pos[eee-1]:e_pos[eee]])[0:6])
        pl.legend(loc='upper left')
        pl.colorbar()
    pl.xlabel('SR in dbz')
    pl.ylabel('GR in dbz')
    pl.grid()
    pl.xlim(0,45)
    pl.ylim(0,45)
    pl.plot([0,45],[0,45], color='black', ls='--')

pl.tight_layout()
##pl.savefig('/automount/ftp/velibor/validation/corr_elev.png')

In [ ]:
xyz_v1[:,:,:,2].shape


In [28]:
import numpy as np
print ('r:',np.cos(np.deg2rad(7.))*50000/1000.)
print ('h:' ,np.sin(np.deg2rad(7.))*50000/1000.)

('r:', 49.627307582066095)
('h:', 6.093467170257374)


In [ ]:
500./0.02

In [29]:
4000./250

16.0

In [31]:
(26**2)/25.

27.04

In [1]:
75*1000

75000

In [11]:
4000./250.


16.0

In [38]:
np.cos(np.deg2rad(0.))

1.0

In [42]:
np.cos(np.deg2rad(180))**2

1.0

In [37]:
data = np.array([[1., 2., 3., 4., np.nan],[1,2,3,4,np.nan]])
np.count_nonzero(~np.isnan(data))

8

In [39]:
np.cos()

6.123233995736766e-17